# The Monte Carlo Simulation of Radiation Transport

WE will discuss essentiall physics and method to do gamma quanta (photons with high enough energy) radiation transport using Monte Carlo methods. We will covers interactions processes, basics of radiation passing through matter as well as Monte Carlo method and how it helps with radiation propagation. 

## Basic physics

We would cover typical energies and wave length when photons are behaving like a point-like particle interaction with matter.

### Units

Common unit for a photon energy would be electron-volt (eV). This is the kinetic energy electron aquire when it moves in electric field (say, between plates of the capacitor) with potential difference 1Volt. This is very small energy and is equal to about $1.6\times10^{-19}$Joules. Typical energies we are interested inare in the 1keV to 100MeV range.

### Spatial size and wave length

Photons are massless particles, and it is very easy to compute photon "size" which is photon wavelength.
$$ \lambda = \frac{hc}{E\gamma} $$
where $\lambda$ is wavelength, $h$ is Plank's constant, $c$ is speed of light and $E\gamma$ is photon energy. For example, lets compute wavelength for photon with energy 1eV.

In [5]:
h = 6.625e-34
c = 3e8
Eg = 1.0 * 1.6e-19
λ = h*c/Eg
print(f"Photon wavelength = {λ*1.0e9} nanometers")

Photon wavelength = 1242.1875 nanometers


Thus, for 1keV photon we will get wave length about 1.2 nm, and for 1MeV photon we will get wave length about $1.2\times10^{-3}$nm.

FOr comparison, typical atom size is from 0.1nm (He) to 0.4nm (Fr and other heavy). Therefore, for most interactions between photon and atoms in our enery range we could consider it particles, not waves.

## Basics of Monte Carlo methods

Was first introduced by Conte du Buffon, as needle dropping experiment to calculate value of $\pi$. Laplace extended the example of the CduB by 

## Basic Photons Interactions with atoms

There are several interaction processess of photons with media.

### Compton Scattering